In [ ]:
import json
from typing import List

import torch
from PIL import Image
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torch import nn
from torchvision import transforms as T
from tqdm.auto import tqdm

from config import BASIC_CONFIG, FT_NET_CFG, SHAPE_EMBEDDING_CFG
from src.datasets.get_loader import get_train_loader
from src.models.baseline import LitModule

In [6]:
model_checkpoint = ModelCheckpoint(every_n_epochs=1)
early_stopping = EarlyStopping(mode='min', monitor='loss', patience=20)

In [3]:
train_loader = get_train_loader()

/home/dustin/anaconda3/envs/reid/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
net = LitModule(shape_edge_index=SHAPE_EMBEDDING_CFG.EDGE_INDEX,
                shape_pose_n_features=SHAPE_EMBEDDING_CFG.POSE_N_FEATURES,
                shape_n_hidden=SHAPE_EMBEDDING_CFG.N_HIDDEN,
                shape_out_features=SHAPE_EMBEDDING_CFG.OUT_FEATURES,
                shape_relation_layers=SHAPE_EMBEDDING_CFG.RELATION_LAYERS,
                class_num=751,
                r50_stride=FT_NET_CFG.R50_STRIDE,
                r50_pretrained_weight=FT_NET_CFG.PRETRAINED, lr=BASIC_CONFIG.LR)


/home/dustin/anaconda3/envs/reid/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [7]:
trainer = Trainer(accelerator='gpu', max_epochs=60, callbacks=[model_checkpoint, early_stopping])

trainer.fit(model=net, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type           | Params
-----------------------------------------------------
0 | ft_net            | FTNet          | 25.6 M
1 | shape_embedding   | ShapeEmbedding | 300 K 
2 | fusion            | FusionNet      | 2.6 M 
3 | id_classification | Linear         | 769 K 
-----------------------------------------------------
3.7 M     Trainable params
25.6 M    Non-trainable params
29.3 M    Total params
117.005   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]